<a href="https://colab.research.google.com/github/parwisenlared/MasterThesis/blob/master/Image_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/b&w" -d "/content/drive/My Drive/Colab Notebooks"

unzip:  cannot find or open /content/drive/My Drive/Colab Notebooks/b&w, /content/drive/My Drive/Colab Notebooks/b&w.zip or /content/drive/My Drive/Colab Notebooks/b&w.ZIP.


In [ ]:
import glob, os
#os.chdir("..")
print(os.getcwd())

#os.listdir(os.getcwd())
#os.chdir("drive/My Drive/Colab Notebooks/")
#print(os.getcwd())

/content/drive/My Drive/Colab Notebooks


In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("b&wZeroPressure/NeckerCubeDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("b&wZeroPressure/NeckerCubeDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

493


(1204, 1204)

## Define CNN model

In [ ]:
def split_dataset(x,y):

    # train and test set (90% training, 10% test)
    X_train = X[:int(0.9*int(len(X)))]
    y_train = y[:int(0.9*int(len(X)))]
    X_test = X[int(0.9*int(len(X))):]
    y_test = y[int(0.9*int(len(y))):]
    
    # validation set (90% training, 10% validation)
    X_train = X_train[:int(0.9*int(len(X_train)))]
    y_train = y_train[:int(0.9*int(len(y_train)))]
    
    X_val = X_train[int(0.9*int(len(X_train))):]
    y_val = y_train[int(0.9*int(len(y_train))):]
    
    return X_train, y_train, X_test, y_test, X_val, y_val

#len(X_val), len(y_val)



In [ ]:
from keras.utils import to_categorical
def prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val):

    # reshape training sets to prepare for CNN
    X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_test = np.array(y_test)
    
    X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    y_val = np.array(y_val)
    
    # one hot encoding of pixels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_val = to_categorical(y_val)
    
    train_norm = X_train.astype('float32')
    test_norm = X_test.astype('float32')
    val_norm = X_val.astype('float32')

    # normalize to range 0-1
    X_train = train_norm / 255.0
    X_test = test_norm / 255.0
    X_val = val_norm / 255.0
    # return dataset
    
    return X_train, y_train, X_test, y_test, X_val, y_val

In [ ]:
# To check 
X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)

X_train, y_train, X_test, y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)

print(len(X_val), len(y_val),len(X_train), len(y_train), len(X_test), len(y_test))


98 98 974 974 121 121


In [ ]:
#for i in X_test:
  #print("value" , i.shape)

In [ ]:
# test harness for evaluating models on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10, mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from sklearn.metrics import cohen_kappa_score

 
# define cnn model
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 1)))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.3))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dropout(0.5))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  print("Model defined")
  return model
 
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    #pyplot.show()
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig('/content/drive/My Drive/Colab Notebooks/plots/' + filename + '_plot.png')
	pyplot.close()
    
 
# run the test harness for evaluating a model
def run_test_harness(x,y):
    
    X_train, y_train, X_test, y_test, X_val, y_val = split_dataset(X,y)
    
    X_train, y_train, X_test, Y_test, X_val, y_val = prepare_dataset(X_train, y_train, X_test, y_test, X_val, y_val)
    
    print(len(X_val), len(y_val))
          
    model = define_model()
    # fit model
    history = model.fit(X_train, y_train, epochs=10, batch_size=24, validation_data=(X_val, y_val), verbose=2)
    # evaluate model
    _, acc = model.evaluate(X_test, Y_test, verbose=1)
    print('Test Accuracy: %.3f' % (acc * 100.0))

    # Kappa statistic

    yhat_classes = model.predict_classes(X_test, verbose=1)

    y_values = y_test

    kappa = cohen_kappa_score(y_values, yhat_classes)


    print("Kappa value is: ", kappa)
    # learning curves
    summarize_diagnostics(history)

## Run Tests

In [ ]:
run_test_harness(X,y)

98 98
Model defined
Train on 974 samples, validate on 98 samples
Epoch 1/10
 - 243s - loss: 0.8181 - accuracy: 0.5133 - val_loss: 0.6827 - val_accuracy: 0.5306
Epoch 2/10
 - 241s - loss: 0.6814 - accuracy: 0.5739 - val_loss: 0.6630 - val_accuracy: 0.6429
Epoch 3/10
 - 239s - loss: 0.6657 - accuracy: 0.6006 - val_loss: 0.6534 - val_accuracy: 0.6531
Epoch 4/10
 - 238s - loss: 0.6543 - accuracy: 0.6345 - val_loss: 0.6456 - val_accuracy: 0.7245
Epoch 5/10
 - 239s - loss: 0.6498 - accuracy: 0.6304 - val_loss: 0.6318 - val_accuracy: 0.7041
Epoch 6/10
 - 239s - loss: 0.6375 - accuracy: 0.6283 - val_loss: 0.6230 - val_accuracy: 0.7041
Epoch 7/10
 - 240s - loss: 0.6308 - accuracy: 0.6468 - val_loss: 0.5903 - val_accuracy: 0.7653
Epoch 8/10
 - 238s - loss: 0.6206 - accuracy: 0.6468 - val_loss: 0.5862 - val_accuracy: 0.8061
Epoch 9/10
 - 239s - loss: 0.6102 - accuracy: 0.6684 - val_loss: 0.5651 - val_accuracy: 0.8061
Epoch 10/10
 - 244s - loss: 0.5811 - accuracy: 0.6786 - val_loss: 0.5482 - val_a

B&W Zero Pressure Spiral

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("b&wZeroPressure/PentagonSpiralDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("b&wZeroPressure/PentagonSpiralDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

607


(1319, 1319)

In [ ]:
run_test_harness(X,y)

107 107
Model defined
Train on 1068 samples, validate on 107 samples
Epoch 1/10
 - 264s - loss: 0.9302 - accuracy: 0.5103 - val_loss: 0.6921 - val_accuracy: 0.5421
Epoch 2/10
 - 263s - loss: 0.6906 - accuracy: 0.5346 - val_loss: 0.6910 - val_accuracy: 0.5047
Epoch 3/10
 - 268s - loss: 0.6876 - accuracy: 0.5459 - val_loss: 0.6915 - val_accuracy: 0.5047
Epoch 4/10
 - 263s - loss: 0.6862 - accuracy: 0.5459 - val_loss: 0.6888 - val_accuracy: 0.5794
Epoch 5/10
 - 269s - loss: 0.6895 - accuracy: 0.5515 - val_loss: 0.6885 - val_accuracy: 0.5981
Epoch 6/10
 - 263s - loss: 0.6851 - accuracy: 0.5627 - val_loss: 0.6866 - val_accuracy: 0.5327
Epoch 7/10
 - 262s - loss: 0.6762 - accuracy: 0.5749 - val_loss: 0.6840 - val_accuracy: 0.5421
Epoch 8/10
 - 267s - loss: 0.6727 - accuracy: 0.5890 - val_loss: 0.6798 - val_accuracy: 0.5794
Epoch 9/10
 - 262s - loss: 0.6805 - accuracy: 0.5618 - val_loss: 0.6781 - val_accuracy: 0.5888
Epoch 10/10
 - 262s - loss: 0.6773 - accuracy: 0.5777 - val_loss: 0.6754 - v

Grey-scale Zero pressure

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("images copia/greyscaleNoZeroPressure/PentagonSpiralDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("images copia/greyscaleNoZeroPressure/PentagonSpiralDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

607


(1310, 1310)

In [ ]:
run_test_harness(X,y)

107 107
Model defined
Train on 1061 samples, validate on 107 samples
Epoch 1/10
 - 264s - loss: 0.5631 - accuracy: 0.6899 - val_loss: 0.3737 - val_accuracy: 0.8785
Epoch 2/10
 - 260s - loss: 0.2842 - accuracy: 0.8709 - val_loss: 0.2013 - val_accuracy: 0.9439
Epoch 3/10
 - 264s - loss: 0.1575 - accuracy: 0.9472 - val_loss: 0.1037 - val_accuracy: 0.9720
Epoch 4/10
 - 259s - loss: 0.1042 - accuracy: 0.9642 - val_loss: 0.0796 - val_accuracy: 0.9813
Epoch 5/10
 - 263s - loss: 0.0836 - accuracy: 0.9680 - val_loss: 0.0674 - val_accuracy: 0.9720
Epoch 6/10
 - 259s - loss: 0.0805 - accuracy: 0.9680 - val_loss: 0.0668 - val_accuracy: 0.9813
Epoch 7/10
 - 259s - loss: 0.0721 - accuracy: 0.9736 - val_loss: 0.0591 - val_accuracy: 0.9813
Epoch 8/10
 - 263s - loss: 0.0668 - accuracy: 0.9746 - val_loss: 0.0494 - val_accuracy: 0.9813
Epoch 9/10
 - 259s - loss: 0.0643 - accuracy: 0.9764 - val_loss: 0.0707 - val_accuracy: 0.9813
Epoch 10/10
 - 262s - loss: 0.0437 - accuracy: 0.9859 - val_loss: 0.0396 - v

Grey Scale Zero Pressure Cube

In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


# The size of the images that your neural network will use

IMG_SIZE = 200

training_data = []

def create_training_data():
  for file in glob.glob("greyscaleZeroPressure/NeckerCubeDrawings/healthy/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 0])
  
  print(len(training_data))

  for file in glob.glob("greyscaleZeroPressure/NeckerCubeDrawings/PD/" + "*.png"):
      img = load_img(file, color_mode = "grayscale")  # this is a PIL image
      new_image = img_to_array(img)  # this is a Numpy array with shape (200, 200,1)
      training_data.append([new_image, 1]) # class is 1 


create_training_data()

random.shuffle(training_data) #shufles the data so it's not control and patient

X = [] #features
y = [] #labels
#y = np.array(y)

for features, label in training_data:
	X.append(features)
	y.append(label)

len(X), len(y)

609


(1311, 1311)

In [ ]:
run_test_harness(X,y)

107 107
Model defined
Train on 1061 samples, validate on 107 samples
Epoch 1/10
 - 263s - loss: 0.7681 - accuracy: 0.5391 - val_loss: 0.6640 - val_accuracy: 0.6075
Epoch 2/10
 - 259s - loss: 0.6689 - accuracy: 0.5862 - val_loss: 0.6540 - val_accuracy: 0.6636
Epoch 3/10
 - 264s - loss: 0.6513 - accuracy: 0.6041 - val_loss: 0.6373 - val_accuracy: 0.6636
Epoch 4/10
 - 259s - loss: 0.6543 - accuracy: 0.5928 - val_loss: 0.6247 - val_accuracy: 0.7009
Epoch 5/10
 - 263s - loss: 0.6390 - accuracy: 0.6343 - val_loss: 0.6180 - val_accuracy: 0.7103
Epoch 6/10
 - 260s - loss: 0.6253 - accuracy: 0.6305 - val_loss: 0.5954 - val_accuracy: 0.7196
Epoch 7/10
 - 259s - loss: 0.6066 - accuracy: 0.6569 - val_loss: 0.5734 - val_accuracy: 0.6822
Epoch 8/10
 - 264s - loss: 0.6036 - accuracy: 0.6550 - val_loss: 0.5709 - val_accuracy: 0.7196
Epoch 9/10
 - 258s - loss: 0.5997 - accuracy: 0.6560 - val_loss: 0.5683 - val_accuracy: 0.7477
Epoch 10/10
 - 262s - loss: 0.5844 - accuracy: 0.6767 - val_loss: 0.5394 - v